In [ ]:
R.home()

In [ ]:
baizer::pkglib(Seurat, SingleR, celldex, tidyverse, baizer, genogamesh, yaml)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])

In [ ]:
# Seurat readin
if (config[[sample]][['FB']]) {
    Lcount <- Read10X(
        data.dir = str_c(snakemake@input[['count_dir']], str_glue(config[['count_mRNA_dir']]), sep='/')
    )
    Mcount <- Lcount[['Gene Expression']]
} else {
    Mcount <- Read10X(
        data.dir = str_c(snakemake@input[['count_dir']], str_glue(config[['count_mRNA_dir']]), sep='/')
    )
}

Mcount <- Mcount[rowSums(Mcount) > 10, colSums(Mcount) > 10]
SE <- CreateSeuratObject(as.data.frame(Mcount), project=sample, 
                         min.cells=config[['Nmin_cell_to_keep_gene']],
                         min.features=config[['Nmin_gene_to_keep_cell']])
SE@meta.data <- SE@meta.data %>% rename(batch=orig.ident)

In [ ]:
# mitochondria umi percent
SE@meta.data[,'mt_percent'] <- PercentageFeatureSet(SE, pattern = "^mt-")

In [ ]:
# cell type
SEref <- readRDS(config[['celltype_ref']])[[config[['celltype_ref_tissue']]]]
SE <- SingleR_SE(SE, SEref)

In [ ]:
# reduction
SE <- reduction_SE(SE, config[['Ndim_umap_use']])

In [ ]:
# stat

TBmtx <- read_csv(str_c(snakemake@input[['count_dir']], str_glue(config[['count_mRNA_mtx']]), sep='/')) %>% 
    dplyr::rename(item=`Metric Name`, value=`Metric Value`) %>%
    filter(`Library Type` == 'Gene Expression')

In [ ]:
Lstat <- TBmtx %>% distinct(item, value) %>% pivot_wider(names_from='item', values_from='value') %>% select(any_of(c(
    'cells'='Cells', 
    'median_genes'='Median genes per cell', 
    'mean_reads'='Mean reads per cell', 
    'median_umis'='Median UMI counts per cell', 
    'align_rate'='Mapped to genome', 
    'total_reads'='Number of reads', 
    'Q30_barcode'='Q30 barcodes', 
    'Q30_umi'='Q30 UMI',
    'Q30_RNA'='Q30 RNA read'
))) %>% as.list

In [ ]:
# write
dir.create(dirname(snakemake@output[['mRNA_csv']]), recursive = TRUE)
write_excel_csv(SE@meta.data %>% r2c('cell'), snakemake@output[['mRNA_csv']])
saveRDS(SE, snakemake@output[['mRNA_rds']])
write_yaml(Lstat, file=snakemake@output[['mRNA_stat']])

In [ ]:
dir.create(snakemake@output[['stat_dir']], recursive = TRUE)
file.copy(snakemake@output[['mRNA_csv']], 
          str_c(snakemake@output[['stat_dir']], '/', basename(snakemake@output[['mRNA_csv']])), 
          overwrite=TRUE)
file.copy(snakemake@output[['mRNA_stat']], 
          str_c(snakemake@output[['stat_dir']], '/', basename(snakemake@output[['mRNA_stat']])), 
          overwrite=TRUE)